<a href="https://colab.research.google.com/github/nnilayy/sec-parser/blob/main/00_mdna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting up

In [ ]:
!pip install sec-parser tiktoken jupyter_black sec_downloader

In [ ]:
from __future__ import annotations
import sec_parser as sp
import sec_downloader as sd
import jupyter_black
import tiktoken

jupyter_black.load()

In [ ]:
!git clone https://github.com/alphanome-ai/sec-parser
%cd "/content/sec-parser"

In [ ]:
!pip install poetry
!poetry export -f requirements.txt --output requirements.txt
!pip install -r "/content/sec-parser/requirements.txt"

## Ticker

In [ ]:
ticker="INTC"

## Ticker Data

In [ ]:
# Download HTML
dl = sd.Downloader("alphanome.ai", "info@alphanome.ai")
html = dl.get_filing_html(form="10-Q",ticker=ticker)
elements = sp.Edgar10QParser().parse(html)
table_elements=[e for e in elements if isinstance(e,sp.TableElement)]
len(table_elements)

51

## HTML

In [ ]:
import re

pattern = r"\[Table with \d+ rows, \d+ numbers, and \d+ characters.]"
matches = re.findall(pattern, markdown)
for i, match in zip(range(len(matches)),matches):
    print(str(i)+" "+match)

In [ ]:
from bs4 import BeautifulSoup
index=0
html=table_elements[index].html_tag.get_source_code()
soup=BeautifulSoup(html,'html.parser')
soup

<div style="margin-bottom:1pt;text-align:center"><table style="border-collapse:collapse;display:inline-table;margin-bottom:5pt;vertical-align:text-bottom;width:94.125%"><tr><td style="width:1.0%"></td><td style="width:28.653%"></td><td style="width:0.1%"></td><td style="width:1.0%"></td><td style="width:22.557%"></td><td style="width:0.1%"></td><td style="width:1.0%"></td><td style="width:13.268%"></td><td style="width:0.1%"></td><td style="width:0.1%"></td><td style="width:0.525%"></td><td style="width:0.1%"></td><td style="width:1.0%"></td><td style="width:30.397%"></td><td style="width:0.1%"></td></tr><tr><td colspan="3" style="padding:0 1pt"></td><td colspan="3" style="padding:2px 1pt;text-align:left;vertical-align:bottom"><span style="color:#262626;font-family:'Arial',sans-serif;font-size:9pt;font-weight:400;line-height:100%;text-decoration:underline"><ix:nonnumeric contextref="c-1" format="ixt-sec:stateprovnameen" id="f-7" name="dei:EntityIncorporationStateCountryCode">Delaware</

## OUTPUT

In [ ]:
from sec_parser.semantic_elements.table_element.table_parser import TableParser
index=0
table=table_elements[index].html_tag.get_source_code()
df=TableParser(table).parse_as_df()
df

,1,2,3,4
0,,Delaware,,94-1672743
1,(State or other jurisdiction of incorporation ...,(State or other jurisdiction of incorporation ...,(State or other jurisdiction of incorporation ...,(I.R.S. Employer Identification No.)
2,"2200 Mission College Boulevard,","Santa Clara,",California,95054-1549
3,(Address of principal executive offices),(Address of principal executive offices),(Address of principal executive offices),(Zip Code)


In [ ]:
import pandas as pd
from IPython.display import HTML

# Sample DataFrame
data = {'A': ['Content A1', 'Content A2', 'Content A2', 'Content A3'],
        'B': ['Content B1', 'Content A2', 'Content B2', 'Content C3'],
        'C': ['Content C1', 'Content A2', 'Content B2', 'Content C3']}
df = pd.DataFrame(data)

# Function to create HTML for merged cells
def merge_cells_html(row):
    result = []

    for key, group in groupby(row):
        values = list(group)
        if len(values) == 1:
            result.append(f'<tr><td>{values[0]}</td></tr>')
        else:
            result.append(f'<td colspan="{len(values)}">{values[0]}</td>')

    return ''.join(result)

# Create a new DataFrame with the 'HTML' column
html_content = df.apply(merge_cells_html, axis=1)
html_table = f'<table>{"".join(html_content)}</table>'

# Use pd.read_html to create a DataFrame from HTML
df_from_html = pd.read_html(html_table)

# Display the original DataFrame and the DataFrame created from HTML
df_from_html

[            0
 0  Content A1
 1  Content B1
 2  Content C1
 3  Content A2
 4  Content A3]

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

html_content = '''

<table>
<tr><th>A</th><th>B</th><th>C</th></tr>
<tr><td>Content A1</td><td>Content B1</td><td>Content C1</td></tr>
<tr><td colspan="3">Content A2</td></tr>
<tr><td>Content A2</td><td colspan="2">Content B2</td></tr>
<tr><td>Content A3</td><td colspan="2">Content C3</td></tr>
</table>
'''

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Use pd.read_html to convert HTML tables to a list of DataFrames
df = pd.read_html(str(soup))[0]

styled_df = df.style.set_table_styles([
    {'selector': 'tr', 'props': [('border', '1px solid grey'),('padding', '8px') , ('text-align', 'left')]},
    {'selector': 'td', 'props': [('border', '1px solid grey'), ('padding', '8px'), ('text-align', 'left')]}
])

# Display the styled DataFrame
styled_df


,A,B,C
0,Content A1,Content B1,Content C1
1,Content A2,Content A2,Content A2
2,Content A2,Content B2,Content B2
3,Content A3,Content C3,Content C3


In [ ]:
import pandas as pd
from itertools import groupby

# Sample DataFrame
data = {'A': ['Content A1', 'Content A2', 'Content A2', 'Content A3'],
        'B': ['Content B1', 'Content A2', 'Content B2', 'Content C3'],
        'C': ['Content C1', 'Content A2', 'Content B2', 'Content C3']}
df = pd.DataFrame(data)

# Drop empty columns

# Function to create HTML for merged cells
def merge_cells_html(row):
    result = []

    for key, group in groupby(row):
        values = list(group)
        if len(values) == 1:
            result.append(f'<td>{values[0]}</td>')
        else:
            result.append(f'<td colspan="{len(values)}">{values[0]}</td>')

    return ''.join(result)

# Create a new DataFrame with the 'HTML' column
html_content = df.apply(merge_cells_html, axis=1)
html_df = pd.DataFrame(html_content)

html_df = html_df.dropna(axis=1, how='all')
# Create the header row
header_row = ''.join([f'<th>{col}</th>' for col in df.columns])

# Concatenate the header row and the HTML content
html_df = pd.concat([pd.DataFrame([header_row]), html_df])
styled_df = html_df.style.set_table_styles([

    {'selector': 'tr', 'props': [('border', '1px solid grey'),('padding', '8px') , ('text-align', 'left')]},
    {'selector': 'td', 'props': [('border', '1px solid grey'), ('padding', '8px'), ('text-align', 'left')]}
])

styled_df

## Preprocessing Functions

In [ ]:
import pandas as pd
from io import StringIO
import re

def basic_preprocessing(html: str) -> pd.DataFrame:
  tables = pd.read_html(StringIO(html), flavor="lxml")
  if len(tables) == 0:
      msg = "No tables found"
      raise ValueError(msg)
  table = tables[0]
  table = table.dropna(how="all")
  table.columns = pd.Index(table.iloc[0].tolist())
  table = table[1:]
  table = table.dropna(axis=1, how="all")
  first_row = list(table.columns)
  new_df = pd.DataFrame([first_row], columns=table.columns)
  table = pd.concat([new_df, table], ignore_index=True)
  table.columns = pd.Index(
      list(range(1, len(table.columns) + 1)),
      dtype=str,
  )
  table = table.fillna("")
  table=table.astype(str)
  table = table.applymap(lambda x: x[:-2] if isinstance(x, str) and x.endswith('.0') else x)
  return table


def remove_blank_columns(df: pd.DataFrame) -> pd.DataFrame:
    columns_to_remove = []
    for i, _ in enumerate(df.columns):
        if i < len(df.columns) - 1:
            current_column = df.columns[i]
            next_column = df.columns[i + 1]
            non_empty_rows = df[
                (df[current_column] != "") & (df[next_column] != "")
            ]
            if (
                non_empty_rows[current_column] == non_empty_rows[next_column]
            ).all():
                if (df[current_column] == "").sum() >= (
                    df[next_column] == ""
                ).sum():
                    columns_to_remove.append(current_column)
                else:
                    columns_to_remove.append(next_column)
    return df.drop(columns=columns_to_remove)


def merge_columns_by_markers(table: pd.DataFrame, markers: list) -> pd.DataFrame:
    for marker in markers:
        marker_columns = [col for col in table.columns if any(str(value) == marker for value in table[col])]

        for current_column in marker_columns:
            current_column_index = table.columns.get_loc(current_column)

            if marker == '$' and current_column_index < len(table.columns) - 1:
                left_column=current_column
                right_column = table.columns[current_column_index + 1]

            elif (marker == '%' or marker == ')' or marker == 'pts' or marker == 'pts.' or marker == '*' or marker == '**') and current_column_index > 0:
                left_column = table.columns[current_column_index - 1]
                right_column=current_column
            else:
                continue

            non_marker_rows = table[(table[left_column] != marker) & (table[right_column] != marker)]
            non_empty_non_marker_rows = non_marker_rows[(non_marker_rows[left_column] != "") & (non_marker_rows[right_column] != "")]
            marker_rows = table.drop(non_empty_non_marker_rows.index)
            marker_rows_indices = marker_rows.index

            if (non_empty_non_marker_rows[left_column] == non_empty_non_marker_rows[right_column]).all():
                table.loc[marker_rows_indices, left_column] = (table.loc[marker_rows_indices, left_column].str.cat(table.loc[marker_rows_indices, right_column], sep="", na_rep="").str.strip())
                table = table.drop(columns=[right_column])

    return table


# -------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------------------------------

def assign_same_value(df):
    first_column = df.columns[0]
    for index, row in df.iterrows():
        if row[first_column] == '' and all(value == row.iloc[1] for value in row.iloc[1:]):
            value_to_assign = row.iloc[1]
            df.loc[index, first_column] = value_to_assign
    return df


def clean_repeating_rows(df):
  same_values_mask = df.apply(lambda row: all(value == '' or value == row.iloc[0] for value in row), axis=1)
  df.loc[same_values_mask, df.columns[1:]] = ''
  return df


def row_splitter(df):
    df['Marked'] = (df.apply(lambda row: any(bool(re.search(r'\$\s*\(\s*[\d,]+\s*\)|\$\s*[\d,]+|\(\s*[\d,]+\s*%\s*\)|[\d,]+\s*%',
                                                            str(cell).replace(" ", ""))) for cell in row), axis=1))
    first_true_index = df.index[df['Marked']].min()
    df=df.drop(columns=["Marked"])
    header_rows = df.iloc[:first_true_index, :]
    df_remaining = df.iloc[first_true_index:, :]
    return header_rows, df_remaining


def redundant_adjacent_row_cleaner(column):
    for i in range(1, len(column)):
        if column[i - 1] == column[i]:
            column[i - 1] = ''
    return column

# -------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------------------------------
def column_to_index(df):
  first_column = df.columns[0]
  column_values = df[first_column].values
  df=df.set_index(column_values)
  df=df.drop(columns=[first_column])
  return df


def multi_indexer(header_rows,df_remaining):
  header_rows=column_to_index(header_rows)
  df_remaining=column_to_index(df_remaining)

  num_rows=len(header_rows)
  rows_list = [header_rows.iloc[i].tolist() for i in range(num_rows)]
  tupled_indices = list(zip(*rows_list))
  # level_names = [f"lvl-{i}" for i in range(num_rows)]
  multi_index_columns=pd.MultiIndex.from_tuples(tupled_indices, names=header_rows.index)
  df_remaining.columns=multi_index_columns
  # df_remaining=df_remaining.reset_index(drop=True)
  return df_remaining

## Output

In [ ]:
based_df=basic_preprocessing(table)
based_df=remove_blank_columns(based_df)
based_df=merge_columns_by_markers(based_df,["$",")","%","pts","pts.","*","**"])
based_df.columns = range(1, len(based_df.columns) + 1)
based_df

In [ ]:
# Splitting df into header and remaining rows
header_rows,df_remaining=row_splitter(based_df)

# Cleaning Header Rows
header_rows=header_rows.apply(redundant_adjacent_row_cleaner, axis=0)

# Cleaning Remaining Rows
df_remaining=assign_same_value(df_remaining)
df_remaining=clean_repeating_rows(df_remaining)
#
#
#
#
data=multi_indexer(header_rows,df_remaining)
data

,1,2,3,4
0,,Delaware,,94-1672743
1,(State or other jurisdiction of incorporation ...,(State or other jurisdiction of incorporation ...,(State or other jurisdiction of incorporation ...,(I.R.S. Employer Identification No.)
2,"2200 Mission College Boulevard,","Santa Clara,",California,95054-1549
3,(Address of principal executive offices),(Address of principal executive offices),(Address of principal executive offices),(Zip Code)


In [ ]:
df.to_json()

In [ ]:
df = df.apply(redundant_adjacent_row_cleaner, axis=0)
(df)

## Future

In [ ]:
# def contains_subtables(df):
# def is_numeric_table(df):
# def correct_index(df):
# cvx-4 works for no index names, something else is the issue